In [26]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from math import sqrt
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import CoClustering, Dataset, Reader
from surprise.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error




## using movielens small dataset




In [ ]:
df1 = pd.read_csv(r"C:\Users\91798\Desktop\flame\FSP\courses'\ML\ml-latest-small\ml-latest-small\ratings.csv")
df1
df1.set_index(['userId', 'movieId'])
df1.info()

In [28]:
df2 = pd.read_csv(r"C:\Users\91798\Desktop\flame\FSP\courses'\ML\ml-latest-small\ml-latest-small\movies.csv")
df2.set_index('movieId')
df2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [29]:
# code to create a genres dataframe 
genresdf = pd.DataFrame()
# finding all genres of each movie and creating genres dictionary
for index, r in df2.iterrows():
    genre_dict = {  "movieId": r['movieId'],
                    "title": r['title'],
                    "Action": False,
                    "Adventure": False,
                    "Animation": False,
                    "Children": False,
                    "Comedy": False,
                    "Crime": False,
                    "Documentary": False,
                    "Drama": False,
                    "Fantasy": False,
                    "Film-Noir": False,
                    "Horror": False,
                    "Musical": False,
                    "Mystery": False,
                    "Romance": False,
                    "Sci-Fi": False,
                    "Thriller": False,
                    "War": False,
                    "Western": False,
                 "IMAX":False}
    
    genres = r['genres'].split("|")
    if "(no genres listed)" not in genres:
        for gen in genres:
            genre_dict[gen] = True
    # appending dictionary values of each row to genres dataframe
    genresdf = genresdf.append(genre_dict, ignore_index=True)

C:\Users\91798\AppData\Local\Temp\ipykernel_26620\1825947147.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  genresdf = genresdf.append(genre_dict, ignore_index=True)
C:\Users\91798\AppData\Local\Temp\ipykernel_26620\1825947147.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  genresdf = genresdf.append(genre_dict, ignore_index=True)
C:\Users\91798\AppData\Local\Temp\ipykernel_26620\1825947147.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  genresdf = genresdf.append(genre_dict, ignore_index=True)
C:\Users\91798\AppData\Local\Temp\ipykernel_26620\1825947147.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  genresdf = gen

In [6]:
genresdf

,movieId,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,IMAX
0,1,Toy Story (1995),False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji (1995),False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men (1995),False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,4,Waiting to Exhale (1995),False,False,False,False,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False
4,5,Father of the Bride Part II (1995),False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),True,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9738,193583,No Game No Life: Zero (2017),False,False,True,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9739,193585,Flint (2017),False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9740,193587,Bungo Stray Dogs: Dead Apple (2018),True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
# Now adding ratings dataframe and movie genres dataframe into a single dataframe
df= df1.merge(genresdf,  on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,IMAX
0,1,1,4.0,964982703,Toy Story (1995),False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,5,1,4.0,847434962,Toy Story (1995),False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,7,1,4.5,1106635946,Toy Story (1995),False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,15,1,2.5,1510577970,Toy Story (1995),False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,17,1,4.5,1305696483,Toy Story (1995),False,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [9]:
df.shape

(100836, 24)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()
#no missing values

In [ ]:
display(df.describe().T)


In [ ]:
df['Crime'].describe()

In [ ]:
(df['rating']).describe().T

## Using KNN

In [55]:

# Standardize the ratings
#scaler = StandardScaler()
#df['rating'] = scaler.fit_transform(df['rating'].values.reshape(-1, 1))
# Pivot the data to have users as rows and movies as columns
df_pivot = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)




In [56]:
# Train the KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(df_pivot)



NearestNeighbors(algorithm='brute', metric='cosine')

In [57]:
# Get the indices of the k most similar users
k = 10 # Change this to the desired number of recommendations
user_index = 1 # Change this to the desired user ID
distances, indices = model_knn.kneighbors(df_pivot.iloc[user_index-1,:].values.reshape(1, -1), n_neighbors=40)

# Get the movie recommendations for the user
movie_ids = df_pivot.columns[indices.flatten()[1:]]
movie_ratings = df_pivot.iloc[user_index-1,:][movie_ids]
movie_recommendations = pd.DataFrame({'movieId': movie_ids, 'predicted_rating': movie_ratings.values})

# Print the top 10 movie recommendations for the user
print('Top {} movie recommendations for user {}:'.format(k, user_index))
print(movie_recommendations.head(10))



Top 10 movie recommendations for user 1:
   movieId  predicted_rating
0      305               0.0
1      354               0.0
2      423               3.0
3       63               0.0
4      102               0.0
5      535               0.0
6       42               0.0
7      329               0.0
8      516               0.0
9       48               0.0


In [58]:

# Get the predictions for all users
distances, indices = model_knn.kneighbors(df_pivot)
all_predictions = []
for i in range(len(df_pivot)):
    if len(indices[i]) > 1:
        user_ratings = df_pivot.iloc[indices[i][1:],:]
        user_predictions = user_ratings.mean(axis=0)
    else:
        user_predictions = pd.Series(np.zeros_like(df_pivot.iloc[0]), index=df_pivot.columns)
    all_predictions.append(user_predictions)

# Calculate the RMSE
rmse = sqrt(mean_squared_error(df_pivot.values.flatten(), pd.concat(all_predictions).values))
print('RMSE:', rmse)

RMSE: 0.4516821584228518


## SVD-Singular value decomposition

In [35]:

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Split the dataset into train and test sets
trainset = data.build_full_trainset()

# Create an SVD algorithm instance
algo = SVD(n_epochs=25, verbose=True)

# Fit the algorithm to the training set
algo.fit(trainset)



Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24


In [36]:
# Get the top 10 movie recommendations for user 1
user_id = 1
movie_ids = [i for i in range(1,df['movieId'].nunique()+1)]
movies_rated_by_user = df[df['userId'] == user_id]['movieId']
movie_ids_to_pred = list(set(movie_ids)-set(movies_rated_by_user))

predicted_ratings = []
for movie_id in movie_ids_to_pred:
    predicted_rating = algo.predict(str(user_id), str(movie_id)).est
    predicted_ratings.append((movie_id, predicted_rating))

recommended_movies = sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:10]
recommended_movies = df[df['movieId'].isin([movie[0] for movie in recommended_movies])][['movieId', 'title']].drop_duplicates().set_index('movieId').loc[[movie[0] for movie in recommended_movies]]['title']
print(recommended_movies)



movieId
2                         Jumanji (1995)
4               Waiting to Exhale (1995)
5     Father of the Bride Part II (1995)
7                         Sabrina (1995)
8                    Tom and Huck (1995)
9                    Sudden Death (1995)
10                      GoldenEye (1995)
11        American President, The (1995)
12    Dracula: Dead and Loving It (1995)
13                          Balto (1995)
Name: title, dtype: object


In [37]:
# Evaluate the algorithm using RMSE and MAE
results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Print the mean RMSE and MAE scores across all folds
print("Mean RMSE score:", round(results['test_rmse'].mean(), 3))
print("Mean MAE score:", round(results['test_mae'].mean(), 3))


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 0
P

## Co-clustering

In [38]:


# Define the reader for the Surprise library
reader = Reader(rating_scale=(0.5, 5.0))

# Load the dataset into the Surprise format
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2)



In [39]:
# Train the Co-clustering model
model_cocluster = CoClustering()
model_cocluster.fit(trainset)

# Get the top k movie recommendations for a given user
k = 10 # Change this to the desired number of recommendations
user_id = 20 # Change this to the desired user ID

# Get the movies that the user has already rated
user_movies = df[df['userId'] == user_id]['movieId'].tolist()

# Get the movies that the user has not rated
anti_testset = trainset.build_anti_testset()
user_anti_testset = filter(lambda x: x[0] == user_id, anti_testset)
predictions = model_cocluster.test(user_anti_testset)



In [40]:
# Sort the predictions by predicted rating in descending order
top_k = sorted(predictions, key=lambda x: x.est, reverse=True)[:k]

# Print the top k movie recommendations for the user
print('Top {} movie recommendations for user {}:'.format(k, user_id))
for i, prediction in enumerate(top_k):
    movie_id = prediction.iid
    title = df[df['movieId'] == movie_id]['title'].values[0]
    predicted_rating = prediction.est
    print('{}. {} ({:.2f})'.format(i+1, title, predicted_rating))
    
    
predictions = model_cocluster.test(testset)
rmse = accuracy.rmse(predictions)
print('RMSE score for Co-clustering model: {:.4f}'.format(rmse))

Top 10 movie recommendations for user 20:
1. Adam's Rib (1949) (5.00)
2. Bossa Nova (2000) (5.00)
3. Front of the Class (2008) (5.00)
4. Kung Fu Panda: Secrets of the Masters (2011) (5.00)
5. Tom Segura: Mostly Stories (2016) (5.00)
6. Match Factory Girl, The (Tulitikkutehtaan tyttö) (1990) (5.00)
7. Cheburashka (1971) (5.00)
8. Supercop 2 (Project S) (Chao ji ji hua) (1993) (5.00)
9. With a Friend Like Harry... (Harry, un ami qui vous veut du bien) (2000) (5.00)
10. Wow! A Talking Fish! (1983) (5.00)
RMSE: 0.9503
RMSE score for Co-clustering model: 0.9503


## Neural networks

In [49]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
# Combine the data
combined = pd.concat([train, test], axis=0)

# Encode the user IDs
user_encoder = LabelEncoder()
combined['userId'] = user_encoder.fit_transform(combined['userId'])

# Encode the movie titles
title_encoder = LabelEncoder()
combined['movieId'] = title_encoder.fit_transform(combined['title'])

# Split the combined data back into training and test sets
train = combined.iloc[:train.shape[0], :]
test = combined.iloc[train.shape[0]:, :]



In [50]:
# Scale the ratings
scaler = StandardScaler()
train['rating'] = scaler.fit_transform(train['rating'].values.reshape(-1, 1))
test['rating'] = scaler.transform(test['rating'].values.reshape(-1, 1))

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mse', optimizer=Adam(lr=0.001))

# Define the early stopping callback
early_stopping = EarlyStopping(patience=5, verbose=1)



C:\Users\91798\AppData\Local\Temp\ipykernel_26620\1269107724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['rating'] = scaler.fit_transform(train['rating'].values.reshape(-1, 1))
C:\Users\91798\AppData\Local\Temp\ipykernel_26620\1269107724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['rating'] = scaler.transform(test['rating'].values.reshape(-1, 1))


In [51]:
# Train the model
history = model.fit(train[['userId', 'movieId']], train['rating'], epochs=50, batch_size=128, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss = model.evaluate(test[['userId', 'movieId']], test['rating'])
test_rmse = np.sqrt(test_loss)
print('Test RMSE score: {:.4f}'.format(test_rmse))

# Predict the ratings on the test set
test['predicted_rating'] = scaler.inverse_transform(model.predict(test[['userId', 'movieId']]))
test['movie_title'] = test['title']

# Group the predicted ratings by movie title and calculate the average rating
avg_ratings = test[['movie_title', 'predicted_rating']].groupby('movie_title').mean().reset_index()

# Print the recommended movies
print('Recommended movies:')
for _, row in avg_ratings.sort_values(by='predicted_rating', ascending=False).iloc[:10].iterrows():
    print('{} ({:.4f})'.format(row['movie_title'], row['predicted_rating']))

Epoch 1/50


C:\Anaconda\envs\STAT321\lib\site-packages\keras\engine\data_adapter.py:1700: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


568/568 [==============================] - 4s 4ms/step - loss: 190431.5312 - val_loss: 0.9906
Epoch 2/50
568/568 [==============================] - 2s 4ms/step - loss: 1527.5686 - val_loss: 0.9906
Epoch 3/50
568/568 [==============================] - 2s 4ms/step - loss: 554.6038 - val_loss: 0.9906
Epoch 4/50
568/568 [==============================] - 2s 4ms/step - loss: 282.8242 - val_loss: 0.9906
Epoch 5/50
568/568 [==============================] - 2s 4ms/step - loss: 183.1531 - val_loss: 0.9906
Epoch 6/50
568/568 [==============================] - 2s 4ms/step - loss: 103.5569 - val_loss: 0.9906
Epoch 7/50
568/568 [==============================] - 2s 4ms/step - loss: 84.4365 - val_loss: 0.9906
Epoch 8/50
568/568 [==============================] - 3s 4ms/step - loss: 43.4981 - val_loss: 0.9906
Epoch 9/50
568/568 [==============================] - 3s 5ms/step - loss: 39.6693 - val_loss: 0.9907
Epoch 9: early stopping
631/631 [==============================] - 2s 3ms/step - loss: 0.984

C:\Users\91798\AppData\Local\Temp\ipykernel_26620\49834315.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_rating'] = scaler.inverse_transform(model.predict(test[['userId', 'movieId']]))
C:\Users\91798\AppData\Local\Temp\ipykernel_26620\49834315.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['movie_title'] = test['title']
